In [1]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import fitz
from pathlib import Path
import torch
from typing import Literal, Any
import re
from extracted_content import LLM_Extracted_content
from utils import get_document_maps
import pandas as pd
from rag_configuration import CFG

In [3]:
#embedding_model = AutoModel.from_pretrained(CFG.embedding_model_name, trust_remote_code=True,
#                                            torch_dtype=torch.bfloat16)
#embedding_tokenizer = AutoTokenizer.from_pretrained(CFG.embedding_model_name)
#reranker_tokenizer = AutoTokenizer.from_pretrained(CFG.reranker_model_name)
#reranker_model = AutoModelForSequenceClassification.from_pretrained(
#    CFG.reranker_model_name, trust_remote_code=True,
#    torch_dtype='auto'
#)
#embedding_tokenizer.model_max_length = 8192
#reranker_tokenizer.model_max_length = 8192

In [4]:
document_paths: list[str] = [str(path) for path in Path(CFG.data_dir).glob('**/*.pdf')]
document_paths

['data\\001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf',
 'data\\040-012l_S2k_Diagnostik-Therapie-Kohlenmonoxidvergiftung_2021-11.pdf',
 'data\\040-014l_S3_Intensivmedizin_nach_Polytrauma_2024-08.pdf']

In [5]:
index_text = fitz.open(document_paths[2])[6].get_text()
index_text.count('.') / len(index_text), 'inhaltsverzeichnis' in index_text.lower()[:200]
index_text

' \n \n \n \n5 \n \nHERAUSGEBENDE \n1 \nDIE WICHTIGSTEN EMPFEHLUNGEN AUF EINEN BLICK \n2 \nGELTUNGSBEREICH UND ZWECK \n9 \nA. ZIELSETZUNG UND FRAGESTELLUNG \n9 \nB. VERSORGUNGSBEREICH \n9 \nC. PATIENT*INNENZIELGRUPPE \n9 \nD. ADRESSATEN \n9 \nE. WEITERE DOKUMENTE ZU DIESER LEITLINIE \n9 \nZUSAMMENSETZUNG DER LEITLINIENGRUPPE \n11 \nLEITLINIENKOORDINATION/ANSPRECHPARTNER \n11 \nBETEILIGTE FACHGESELLSCHAFTEN UND ORGANISATION \n12 \nKAPITELAUTOREN \n14 \nVORWORT \n17 \nII. HANDLUNGSEMPFEHLUNGEN IM BEREICH INTENSIVMEDIZIN NACH POLYTRAUMA \n18 \n1 INTERDISZIPLINÄRE UND INTERPROFESSIONELLE ZUSAMMENARBEIT UND KOMMUNIKATION \n19 \n1.1 PRINZIPIEN DER ZUSAMMENARBEIT IN DER POLYTRAUMAVERSORGUNG \n19 \n1.2 OPTIMIERTES SETTING FÜR ZUSAMMENARBEIT UND KOMMUNIKATION \n20 \n1.3 BETEILIGE FACHDISZIPLINEN UND PROFESSIONEN IN DER INTENSIVMEDIZINISCHEN POLYTRAUMA-VERSORGUNG 21 \n1.4 NOTWENDIGE RESSOURCEN \n22 \n1.5 PRAKTISCHE DURCHFÜHRUNG DER PATIENT*INNEN-ÜBERGABE NACH POLYTRAUMA \n23 \n1.6 DOKUMENTATIONS

In [6]:
class BaseDocument:
    def __init__(self, pdf_path: str):
        self.pdf_path = pdf_path
        self.title = self.get_title()

    def get_title(self):
        return re.search(CFG.filename_pattern, self.pdf_path).group(1).replace('-', ' ').replace('_', ' ')


class DocumentIndexData(BaseDocument):

    def __init__(self, pdf_path: str, index_page: int, toc_text: str):
        super().__init__(pdf_path)
        self.toc_pages = [index_page]
        self.toc_text = toc_text
        self.chapters = self.get_chapters()
        self.chapter_ranges = self.get_chapter_ranges()
        self.useless_chapter_keywords = self.get_useless_chapter_keywords()

    def add_to_index(self, extra_page: int, extra_text: str):
        self.toc_pages.append(extra_page)
        self.toc_text += '\n' + extra_text

    def __str__(self):
        return f'Document at {self.pdf_path}'

    def __repr__(self):
        return str(self)

    def get_index_range(self) -> tuple[int, int]:
        return self.toc_pages[0], self.toc_pages[-1]

    def get_chapters(self) -> dict[str, int]:
        chapter_mapping = LLM_Extracted_content.contents[str(self.pdf_path).lower().replace('\\', '/')]
        return {key.split(' ', 1)[1] if ' ' in key else key: value for key, value in chapter_mapping.items()}

    def get_chapter_ranges(self) -> dict[str, tuple[int, int]]:
        return {key: (self.chapters[key]-1, next_value-1) for key, next_value in
                zip(self.chapters.keys(), list(self.chapters.values())[1:] + [10 ** 10])}

    @staticmethod
    def get_useless_chapter_keywords() -> list[str]:
        original_useless_chapter_keywords = ['Literaturverzeichnis', 'Inhaltsverzeichnis', 'Literatur',
                                             'Verwendete Abkürzungen']
        useless_chapter_keywords = original_useless_chapter_keywords.copy()
        useless_chapter_keywords += [keyword.lower() for keyword in original_useless_chapter_keywords]
        useless_chapter_keywords += [keyword.upper() for keyword in original_useless_chapter_keywords]
        useless_chapter_keywords += [keyword.capitalize() for keyword in original_useless_chapter_keywords]
        return list(set(useless_chapter_keywords))




In [7]:
documents = []
for doc in document_paths:
    print(doc)
    new_document = None
    for num, page in enumerate(fitz.open(doc)):
        page_text = page.get_text()

        if ('inhaltsverzeichnis' in page_text.lower()[:200] or new_document) and (
                page_text.count('.') / len(page_text) > CFG.index_dot_percentage_threshold):
            if new_document:
                new_document.add_to_index(num, page_text)
            else:
                new_document = DocumentIndexData(doc, num, page_text)
        elif new_document:
            documents.append(new_document)
            break
            #print('\n'+50*'#')
    #print(doc)
    #print('\n'+50*'#')
    #find_toc_text(doc)
    #print('\n\n\n')

documents

data\001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf
data\040-012l_S2k_Diagnostik-Therapie-Kohlenmonoxidvergiftung_2021-11.pdf
data\040-014l_S3_Intensivmedizin_nach_Polytrauma_2024-08.pdf


[Document at data\001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf,
 Document at data\040-012l_S2k_Diagnostik-Therapie-Kohlenmonoxidvergiftung_2021-11.pdf]

**Add missing document manually**

In [8]:
missing_document: fitz.Document = fitz.open(document_paths[-1])
page_range = (6, 10)
if len(documents) < 3:
    for page in range(*page_range):
        try:
            missing_index_data.add_to_index(page, missing_document[page].get_text())
        except NameError:
            missing_index_data = DocumentIndexData(document_paths[-1], page, index_text)
    documents.append(missing_index_data)
documents

[Document at data\001-012l_S3_Analgesie-Sedierung-Delirmanagement-in-der-Intensivmedizin-DAS_2021-08.pdf,
 Document at data\040-012l_S2k_Diagnostik-Therapie-Kohlenmonoxidvergiftung_2021-11.pdf,
 Document at data\040-014l_S3_Intensivmedizin_nach_Polytrauma_2024-08.pdf]

In [9]:
class DocumentChunk(BaseDocument):
    def __init__(self, pdf_path: str, chunk_title: str, chunk_page: int, chunk_text: str):
        super().__init__(pdf_path)
        self.chunk_title = chunk_title
        self.chunk_page = chunk_page
        self.chunk_text = chunk_text

    def get_data(self) -> dict[str, Any]:
        return {'title': self.chunk_title, 'text': self.chunk_text}



In [10]:
documents[0].chapter_ranges

{'Leitsatz': (2, 3),
 'Inhaltsverzeichnis': (3, 4),
 'Analgesie, Sedierung und Delirmanagement bei Erwachsenen': (4, 4),
 'Relevanz': (4, 4),
 'Risikofaktoren in der Intensivmedizin': (4, 7),
 'Prävention und Risikoreduktion': (7, 10),
 'Prävention und Frühintervention akuter Belastungen zur Reduktion von Spätfolgen': (10,
  16),
 'Monitoring von Analgesie, Sedierung, Delir, Stress, Angst und Schlaf': (16,
  165),
 'Therapeutische Konzepte': (165, 37),
 'Nicht-pharmakologisch': (37, 40),
 'Pharmakologisch': (40, 70),
 'Regionale Analgesieverfahren in der Intensivmedizin': (70, 82),
 'Neuromuskuläre Blockade in der Intensivmedizin': (82, 87),
 'Intra- und Interhospitaltransporte': (87, 88),
 'Spezielle Patient:innengruppen': (88, 88),
 'Schwangerschaft und Stillzeit': (88, 98),
 'Sterbende Patient:innen': (98, 100),
 'Brandverletzte Patient:innen': (100, 101),
 'Das brandverletzte Kind': (101, 104),
 'Polytraumatisierte Patient:innen': (104, 107),
 'Patient:innen mit erhöhtem intrakrani

In [11]:
text_data = {'section_title': [], 'document_title': [], 'headline': [], 'text': [], 'text_for_embedding': []}

for doc, toc_data in zip(document_paths, documents):

    sections, section_pages = get_document_maps(doc)

    for headline in sections:
        document_title = toc_data.title
        text = sections[headline]
        text_data['headline'].append(headline)
        text_data['document_title'].append(document_title)
        text_data['text'].append(text)
        try:
            section_title = [key for key, value in toc_data.chapter_ranges.items() if value[0] <= section_pages[headline] <= value[1]][0]
            text_data['section_title'].append(section_title)
                
        except IndexError:
            section_title = ''
            text_data['section_title'].append(section_title)
        text_data['text_for_embedding'] = '\n'.join([document_title, section_title, headline, text])



In [14]:
df = pd.DataFrame(text_data)
df

,section_title,document_title,headline,text,text_for_embedding
0,,S3 Analgesie Sedierung Delirmanagement in der ...,,\n1,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
1,,S3 Analgesie Sedierung Delirmanagement in der ...,"S3-Leitlinie Analgesie, Sedierung und Delirman...",\nAWMF-Registernummer: 001/012,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
2,,S3 Analgesie Sedierung Delirmanagement in der ...,Federführende Fachgesellschaften,\nDeutsche Gesellschaft für Anästhesiologie un...,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
3,,S3 Analgesie Sedierung Delirmanagement in der ...,Beteiligte Fachgesellschaften:,\nDeutsche Gesellschaft für Chirurgie (DGCH)\n...,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
4,,S3 Analgesie Sedierung Delirmanagement in der ...,Ansprechpartner / Leitliniensekretariat,\nKlinik für Anästhesiologie m. S. operative I...,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
...,...,...,...,...,...
486,DER LEITLINIE,S3 Intensivmedizin nach Polytrauma,IX. Verwendete Abkürzungen,\n\nA.S.P.E.N\n\nAmerikanischen Fachgesellscha...,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
487,DER LEITLINIE,S3 Intensivmedizin nach Polytrauma,Versionsnummer: Erstveröffentlichung:,\n,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
488,DER LEITLINIE,S3 Intensivmedizin nach Polytrauma,Nächste Überprüfung geplant:,\n\n07/2024\n07\n/\n2029\nDie AWMF erfasst und...,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...
489,DER LEITLINIE,S3 Intensivmedizin nach Polytrauma,Insbesondere bei Dosierungsangaben sind stets ...,\n,S3 Intensivmedizin nach Polytrauma\nDER LEITLI...


In [15]:
df.document_title.unique()

array(['S3 Analgesie Sedierung Delirmanagement in der Intensivmedizin DAS',
       'S2k Diagnostik Therapie Kohlenmonoxidvergiftung',
       'S3 Intensivmedizin nach Polytrauma'], dtype=object)

In [ ]:
huggingface_dataset = Dataset.from_pandas(df)